In [ ]:
from pathlib import Path

from utilities import get_castep_outputs
from tensile_test import get_tensile_test, show_traction_separation, show_multiple_traction_separation

# Visualise ab-initio tensile test data

**Σ7 tilt A GB**

In [ ]:
tt = {
    'Pristine': get_tensile_test('s7-tlA-gb'),
    'Iodine': get_tensile_test('s7-tlA-gb-a1'),
}
show_multiple_traction_separation(tt)

**Σ7 tilt A bulk**

In [ ]:
tt = {
    'Pristine': get_tensile_test('s7-tlA-b'),
    'Iodine': get_tensile_test('s7-tlA-b-a1'),
}
show_multiple_traction_separation(tt)

**Σ7 twist GB**

In [ ]:
tt = {
    'Pristine': get_tensile_test('s7-tw-gb'),
    'Iodine': get_tensile_test('s7-tw-gb-a1'),
}
show_multiple_traction_separation(tt)

**Σ7 twist bulk**

In [ ]:
tt = {
    'Pristine': get_tensile_test('s7-tw-b'),
    'Iodine': get_tensile_test('s7-tw-b-a1'),
}
show_multiple_traction_separation(tt)

# Visualise structure at a given tensile test coordinate

In [ ]:
tensile_test = get_tensile_test('s7-tlA-gb-a1')
tensile_test.get_coordinate(5).structure.show(layout_args={'height': 800})

# Method of tensile test inputs generation and analysis

This section breaks down the steps used to:

- generate ab-initio tensile test input files (for CASTEP)
- collate the results
- find the traction separation curve
- visualise the traction separation curve

In [ ]:
import copy

import numpy as np
from atomistic.tensile_test import AtomisticTensileTest
from castep_parse import write_input_files as write_castep_input_files

from utilities import get_castep_parameters, get_simulated_bicrystal

In [ ]:
CASTEP_SIMS_DIR = Path('/path/to/simulation/directory')

**Generate an `AtomisticTensileTest`**

In [ ]:
structure_code = 's7-tw-b-a1'
bicrystal = get_simulated_bicrystal(structure_code, opt_idx=-1)

In [ ]:
exp = np.array([

])
exp = np.sort(exp)

In [ ]:
tensile_test = AtomisticTensileTest(
    base_structure=bicrystal,
    expansions=exp,
    distribution='flat',
)

**Iterate over all coordinates to generate simulation input files**

In [ ]:
base_path = CASTEP_SIMS_DIR.joinpath(structure_code)
input_paths = []

common_castep_params = get_castep_parameters(num_atoms=bicrystal.num_atoms)
common_castep_params['atom_constraints'] = None

for i in tensile_test.all_coordinates():
    
    # Make a directory for this sim:
    sim_path = base_path.joinpath(i.coordinate_fmt)
    sim_path.mkdir()
    
    # Write simulation inputs:
    cst_params = {
        'supercell': i.structure.supercell,
        'atom_sites': i.structure.atoms.coords,
        'species': i.structure.atoms.labels['species'].unique_values,
        'species_idx': i.structure.atoms.labels['species'].values_idx,
        'dir_path': sim_path,
        **copy.deepcopy(common_castep_params),
    }
    input_paths.append(write_castep_input_files(**cst_params))
    